In [1]:
import pandas as pd
import s3fs
from sklearn.metrics import mean_squared_error
import pickle
import mlflow

In [2]:
#we read the model read the model


In [3]:
#testing the model in production
#we load the input
def read_dataframe(filename):
    if filename.endswith('.parquet'):
        df = pd.read_parquet(filename)
    return  df    

In [4]:
#creating step for a new file 
#we transform the model to a dictionary
def preparing_features(input_file, dv):

    df_train = read_dataframe(input_file)
    categorical = ['squareMeters']
    df_train_data = df_train[categorical]
    train_dicts = df_train_data[categorical].to_dict(orient='records')
    X_test = dv.transform(train_dicts)
    target = 'price'
    y_test = df_train[target].values

    return X_test , y_test

In [5]:
def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [6]:

    #we import the model and the dict vectorizer
dv, model = pickle.load(open("models/linear-reg",'rb'))

i = 3
input_file = f"s3://mlops-project-dataset-deen/paris-housing-dataset/ParisHousing_period_{i:02d}.parquet"


X_test , y_test = preparing_features(input_file, dv)



/home/deen/.local/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator DictVectorizer from version 1.1.1 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/deen/.local/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LinearRegression from version 1.1.1 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [7]:
#time to show duration
#to check model in production
model_name = "paris-model-registry"
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

MlflowException: Model Registry features are not supported by the store with URI: 'file:///home/deen/Desktop/datatalks/Mlops-project/model_choosing/mlruns'. Stores with the following URI schemes are supported: ['databricks', 'http', 'https', 'postgresql', 'mysql', 'sqlite', 'mssql'].